In [1]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, plot_confusion_matrix,accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Input, Dense
#necessita do pydot pip install pydot
import pydot

###Carga do Dataset

In [16]:
url = 'https://telescopeinstorage.blob.core.windows.net/datasets/DadosSpotify.csv'
dataset = pd.read_csv(url, engine='python')
dataset.head()

,id,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,target,song_title,artist
0,0,0.0102,0.833,204600,0.434,0.021900,2,0.1650,-8.795,1,0.4310,150.062,4.0,0.286,1,Mask Off,Future
1,1,0.1990,0.743,326933,0.359,0.006110,1,0.1370,-10.401,1,0.0794,160.083,4.0,0.588,1,Redbone,Childish Gambino
2,2,0.0344,0.838,185707,0.412,0.000234,2,0.1590,-7.148,1,0.2890,75.044,4.0,0.173,1,Xanny Family,Future
3,3,0.6040,0.494,199413,0.338,0.510000,5,0.0922,-15.236,1,0.0261,86.468,4.0,0.230,1,Master Of None,Beach House
4,4,0.1800,0.678,392893,0.561,0.512000,5,0.4390,-11.648,0,0.0694,174.004,4.0,0.904,1,Parallel Lines,Junior Boys


In [4]:
dataset.shape

(2017, 17)

###Pré-processamento

In [5]:
#Coloque Aqui as fases de pré-processamento
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017 entries, 0 to 2016
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                2017 non-null   int64  
 1   acousticness      2017 non-null   float64
 2   danceability      2017 non-null   float64
 3   duration_ms       2017 non-null   int64  
 4   energy            2017 non-null   float64
 5   instrumentalness  2017 non-null   float64
 6   key               2017 non-null   int64  
 7   liveness          2017 non-null   float64
 8   loudness          2017 non-null   float64
 9   mode              2017 non-null   int64  
 10  speechiness       2017 non-null   float64
 11  tempo             2017 non-null   float64
 12  time_signature    2017 non-null   float64
 13  valence           2017 non-null   float64
 14  target            2017 non-null   int64  
 15  song_title        2017 non-null   object 
 16  artist            2017 non-null   object 


In [17]:
dataset.drop(['id','song_title','artist'], axis=1, inplace =True)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2017 entries, 0 to 2016
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acousticness      2017 non-null   float64
 1   danceability      2017 non-null   float64
 2   duration_ms       2017 non-null   int64  
 3   energy            2017 non-null   float64
 4   instrumentalness  2017 non-null   float64
 5   key               2017 non-null   int64  
 6   liveness          2017 non-null   float64
 7   loudness          2017 non-null   float64
 8   mode              2017 non-null   int64  
 9   speechiness       2017 non-null   float64
 10  tempo             2017 non-null   float64
 11  time_signature    2017 non-null   float64
 12  valence           2017 non-null   float64
 13  target            2017 non-null   int64  
dtypes: float64(10), int64(4)
memory usage: 220.7 KB


In [18]:
X = dataset.drop(['target'], axis = 1).values
y = dataset['target'].values

In [19]:
#Separação do Conjunto de Treinamento e Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle= True, random_state=25)

In [20]:
#Normalização do Conjunto
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
X_train.shape

(1411, 14)

###Classificação

In [44]:
#Modelo da Rede MLP
model = Sequential()
model.add(Dense(25, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [45]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
history  = model.fit(X_train, y_train, epochs=300, batch_size=70)

Epoch 1/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4164 - accuracy: 0.8150
Epoch 2/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4151 - accuracy: 0.8157
Epoch 3/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4139 - accuracy: 0.8136
Epoch 4/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4120 - accuracy: 0.8179
Epoch 5/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4114 - accuracy: 0.8164
Epoch 6/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4105 - accuracy: 0.8172
Epoch 7/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4088 - accuracy: 0.8136
Epoch 8/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4094 - accuracy: 0.8157
Epoch 9/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4062 - accuracy: 0.8157
Epoch 10/300
21/21 [==============================] - 0s 2ms/step - loss: 0.4043 - accuracy: 0.8164
Epoch 11/

In [ ]:
#Processo de treinamento com 70% dos dados
y_pred_out  = model.predict(X_test)
print(y_pred_out)

y_pred_classes = (y_pred_out > 0.5).astype(int) 
y_pred_classes

In [49]:
#Processo de teste com 30% dos dados que não foram utilizados no treinamento
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

19/19 [==============================] - 0s 1ms/step - loss: 0.7580 - accuracy: 0.7178

accuracy: 71.78%


In [ ]:
#Avaliação do Modelo
